#### Lecture 1. Sequence to sequence

- Machine translation : many to many 에 해당
- encoder + decoder
    - 각각은 LSTM RNN model
    - encoder 마지막 step의 hidden state vector 가 decoder 의 h0 로 적용
    - decoder 첫번째 입력은 < SoS >가 주어지고 첫번째 cell부터 예측 수행하고 < EoS > 토큰을 마주치면 생성중지
- Attention
    - RNN 구조에서 벡터의 차원이 유지되므로 많은 정보가 후발 벡터에는 누적되어 초기 정보 유실 가능성
    - 입력 문장의 순서를 뒤집는 트릭도 사용된 적이 있음
    - tf.seq2seq : https://google.github.io/seq2seq/
    - decoder의 첫번째 hidden state vector가 다음 단어 예측에 사용될 뿐 아니라 encoder의 hidden state vector와의 내적을 통해 무엇이 필요한지 선별하는 Attention scores를 생성하여 softmax 확률분포 도출(attention vector) => 가중평균된 벡터 출력 (Attention output) - 특정 time 의 decoder 값과 attention output 을 조합하여 그 time의 output 도출
    - 위의 작업을 반복하여 encoder hidden state vector에 직접 적용한 가중 평균된 벡터를 output layer의 직접적인 입력의 일부로 사용함으로써 해당 time step의 예측의 정확성을 높임
    - Teacher forcing : 학습 중 예측값이 decoder의 입력으로 들어가는 것이 아니라 ground truth 값이 입력
    - teacher forcing 이 학습 속도가 빠르지만 학습 후 실제 사용했을 때 적응하지 못하는 경향 (노출 편향)
    - 학습의 전반부는 teacher forcing // 이후에는 teacher forcing X
    
- 유사도를 구하는 Attention mechanism
    - $ score(h_t, \bar h_s) = h_t^T\bar h_s $ (dot - 내적 : 벡터들끼리 element-wise 성분곱)
    - $ score(h_t, \bar h_s) = h_t^T W_a \bar h_s $ (general : 아닌 가중치 행렬을 두어 다른 순서 성분곱 반영)
    - $ score(h_t, \bar h_s) = v_a^T tanh(W_a[h_t, \bar h_s ])$ <br>
    : concat 기반 attention (hidden state vector 들을 concat 하여 scalar값을 출력으로 하는 fully connected MLP 학습)
- Attention의 효과
    - NMT (Neural Machine Translation) 성능 비약적 향상
    - bottle neck problem 근본적 해결 : decoder에서 직접적으로 소스에 접근
    - gradient vanishing 해결 : shortcut path 생성
    - attention vector를 조사하면 decoder에서 어떤 단어에 주목하는지 파악 가능

#### Lecture 2. Beam Search

- Greedy decoding : 다음 단어만을 예측하는 task를 학습하고, 매 time마다 가장 높은 확률의 단어 하나만을 택해서 decoding을 진행
- 잘못 예측하는 경우 돌아갈 수단이 없어 최적의 예측값을 내어주지 못하는 상황 발생
- 그렇다고 최적의 가능성을 구하기 위해 모든 경우의 수를 구하는 것은 각각의 step t에 대하여 (Vocabulary size)V<sup>t</sup> 만큼의 partial translation을 구해야 하며 O(V<sup>t</sup>)는 너무 큰 사이즈
- Beam Search : 위 둘의 절충안
    - 매 decoder step에서 k개의 most probable 한 partial translation 추적(hypothesis)
    - $score(y_1,...,y_t) = log P_{LM}(y_1,...,y_t|x) = \sum_{i=1}^t log P_{LM}(y_i|y_1,...,y_{i-1},x) $
    - 최선해를 보장할 수는 없지만 적절한 효율
    - 모델이 < END > token을 생성할 때 종료, 각각의 hypothesis마다 다른 시점에 completed
    - 전체 search는 지정된 timestep 또는 완성된 hypothesis 의 개수에 도달하면 종료
    - 길이가 긴 sequence는 score가 낮아질 가능성이 크므로 생성된 time t로 나눠서 보정

#### Lecture 3. BLEU Score

- 자연어 생성 task 의 정확성 판단 척도

- 기존 방법으로 판단이 어려움 (ex "I love you" vs "oh I love you")
- F-measure $ = \frac{precision \times recall}{\frac{1}{2}(precision + recall)} $ 정밀도와 재현율의 조화평균 (둘중 작은값과 가까움)
    - 정밀도 $ \frac{\# (correct words)}{length\_of\_prediction} $
    - 재현율 $ recall = \frac{\#(correct words)}{length\_of\_reference} $
    - 순서만 다른 경우 no penalty

- BiLingual Evaluation Understudy (BLEU)
    - 기계번역 output과 원본 문장과의 N-gram overlap 수행
    - precision만을 고려 (단어의 생략보다 오역에 민감해지기 위해)
    - BLEU = $ min(1,\frac{length\_of\_prediction}{length\_of\_reference})(\prod^4_{i=1} precision)^{\frac{1}{4}} $  기하평균 * (recall penalty)